''' This program runs a dynamic topic model

#Date: Mar 4, 2019

#Author: Carly Knight

#https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb
#https://stackoverflow.com/questions/50413059/dynamic-topic-modeling-with-gensim-which-code'''


## set up

In [1]:
import logging
import os
from gensim import corpora, utils
from gensim.models.wrappers.dtmmodel import DtmModel
import numpy as np
import pandas as pd
from gensim.corpora import Dictionary
from collections import defaultdict



### dtm

In [2]:
# you can also copy the path down directly. Change this variable to your DTM executable before running.
dtm_path = "/usr/local/bin/dtm-darwin64"

# Open corpus

In [3]:
file_loc = "/Users/carlyknight/Documents/Data/Annual Report/report_paragraphs/future_texts/"

#limit to over 1930
metadata = pd.read_csv(file_loc + "metadata_futureperfect.csv") 
metadata = metadata[(metadata['Year'] >= 1930)]
metadata['text_filename'] = metadata['Filename'].str.replace(".xml", ".txt")

#pick relevant files
textfiles = [i for i in metadata['text_filename'].values.tolist()]

In [4]:
#open files
for index, row in metadata.iterrows():
    txtfile= row['Filename'].replace(".xml", ".txt")
    #read in text file and turn into new variable
    with open(file_loc + txtfile, 'r') as f:
        metadata.loc[index,'text']= f.read()
        f.close()

In [5]:
#metadata.shape (36479,17)

In [6]:
#remove puncutation
#metadata['text'] = metadata['text'].str.replace(r'[^\w\s]+', '')

#remove numbers
#metadata['text'] = metadata['text'].str.replace(r'[\d]+', '')

#remove extraspaces
#metadata['text'] = metadata['text'].str.replace(r'\s+', ' ')

#turn to text
documents = metadata['text'].tolist()

## Preprocess

In [7]:
from gensim.parsing.preprocessing import preprocess_string

#processed corpus: removes punctuation, whitespaces, stopwords, stems, numbes
processed_corpus = [preprocess_string(document) for document in documents] 

#remove low frequency terms (terms with fewer than 20) (brings it down to 35,244 from 953,010 tokens)
frequency = defaultdict(int)

for text in processed_corpus:
    for token in text:
        frequency[token] += 1

processed_corpus = [
    [token for token in text if frequency[token] > 20]
    for text in processed_corpus
]

#dictionary
dictionary = Dictionary(processed_corpus)

#filter extremes: get rid of words that appear in no fewer than 10 documents
dictionary.filter_extremes(no_below=15)
len(dictionary) #33,025

#Bag of Words
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

#Time slice
counts = metadata[['Year', "Filename"]].groupby(['Year']).agg(['count'])
timeslice=[i[0] for i in counts.values.tolist()]

# DTM

In [ ]:
model = DtmModel(dtm_path, bow_corpus, timeslice, num_topics=70,
                 id2word=dictionary, initialize_lda=True)

In [90]:
topics = model.show_topic(topicid=1, time=10, topn=30)
topics

[(0.01306688577931387, 'att'),
 (0.011909793220351062, 'fcc'),
 (0.011047612470614126, 'network'),
 (0.010441001555762832, 'commun'),
 (0.010253314866667177, 'revenu'),
 (0.00869561702492049, 'directv'),
 (0.0083704863324877, 'satellit'),
 (0.00816489166924309, 'wireless'),
 (0.007989921043732216, 'liberti'),
 (0.006754653406689982, 'system'),
 (0.006684133533246968, 'motorola'),
 (0.006452662712663201, 'local'),
 (0.006259950418967462, 'digit'),
 (0.006075097173693343, 'cabl'),
 (0.005966599844226008, 'access'),
 (0.00539191445340964, 'group'),
 (0.005352040750421336, 'internet'),
 (0.005090304519317974, 'hotel'),
 (0.004944988402084844, 'sprint'),
 (0.00473337956054552, 'offer'),
 (0.004592451409420068, 'telecommun'),
 (0.00426048963297498, 'connect'),
 (0.00420312018354661, 'competit'),
 (0.004194116304890932, 'alltel'),
 (0.004027564185158467, 'billion'),
 (0.003969594091958759, 'technolog'),
 (0.0038930195185951397, 'properti'),
 (0.0038228302663059228, 'broadband'),
 (0.003736473

In [19]:
#https://towardsdatascience.com/topic-modeling-with-gensim-a5609cefccc

# save

In [91]:
import pickle

In [92]:
f = open('/Users/carlyknight/Documents/Data/FuturePerfect/dynamic_topic_models/topic70.pkl', 'wb') 
pickle.dump(model, f , protocol=4)          
f.close()                 